In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('data\Clean_Dataset.csv')

In [5]:
#Encode the ordinal variables "stops" and "class".
df["stops"] = df["stops"].replace({'zero':0,'one':1,'two_or_more':2}).astype(int)
df["class"] = df["class"].replace({'Economy':0,'Business':1}).astype(int)

#Create the dummy variables for the cities, the times and the airlines.
dummies_variables = ["airline", "source_city", "destination_city", "departure_time", "arrival_time"]
dummies = pd.get_dummies(df[dummies_variables], drop_first= True)
df = pd.concat([df, dummies], axis=1)

#Create the dummy variables for the cities, the times and the airlines.
df = df.drop(["Unnamed: 0", "flight", "airline", "source_city", "destination_city", "departure_time", "arrival_time"], axis=1)
df.head()

,stops,class,duration,days_left,price,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,...,departure_time_Early_Morning,departure_time_Evening,departure_time_Late_Night,departure_time_Morning,departure_time_Night,arrival_time_Early_Morning,arrival_time_Evening,arrival_time_Late_Night,arrival_time_Morning,arrival_time_Night
0,0,0,2.17,1,5953,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,2.33,1,5953,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,0,0,2.17,1,5956,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,0,0,2.25,1,5955,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,2.33,1,5955,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in df.columns:
    if df[col].dtype=='object':
        df[col]=le.fit_transform(df[col])

In [11]:
len(df.flight.value_counts())

1561

In [13]:
df.drop(['Unnamed: 0', 'flight'], axis=1, inplace=True)

In [14]:
df.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,4,2,2,2,5,5,1,2.17,1,5953
1,4,2,1,2,4,5,1,2.33,1,5953
2,0,2,1,2,1,5,1,2.17,1,5956
3,5,2,4,2,0,5,1,2.25,1,5955
4,5,2,4,2,4,5,1,2.33,1,5955


In [15]:
from sklearn.preprocessing import MinMaxScaler

names = df.columns
indexes = df.index

mmscaler=MinMaxScaler(feature_range=(0,1))
df = mmscaler.fit_transform(df)

data_scaled = pd.DataFrame(df, columns=names, index=indexes)
data_scaled.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0.8,0.4,0.4,1.0,1.0,1.0,1.0,0.027347,0.0,0.039749
1,0.8,0.4,0.2,1.0,0.8,1.0,1.0,0.030612,0.0,0.039749
2,0.0,0.4,0.2,1.0,0.2,1.0,1.0,0.027347,0.0,0.039773
3,1.0,0.4,0.8,1.0,0.0,1.0,1.0,0.028980,0.0,0.039765
4,1.0,0.4,0.8,1.0,0.8,1.0,1.0,0.030612,0.0,0.039765


In [17]:
X = data_scaled.drop('price', axis=1)
y = data_scaled['price']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = SVR()

X_train.shape, X_test.shape, y_test.shape, y_test.shape

((210107, 9), (90046, 9), (90046,), (90046,))

In [19]:
%%time
model.fit(X_train, y_train)

CPU times: total: 24min 45s
Wall time: 25min 39s


SVR()

In [20]:
import pickle


def serialize_model(name_model, model):
    with open('C:/Users/seito/.vscode/Python Practicum/data'+name_model+'2.pkl', 'wb') as file: # wb stands for write binary and open is used to serialize the model
        pickle.dump(model, file) # dump is used to serialize the ML model

serialize_model('model_SVR', model)

In [21]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)

print(r2_score(y_test, y_pred))

0.9071460942687499
